# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=simous password=Sommer69")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
!pwd

/Users/home/Documents/Code/Udacity/Data_Engineering/Sparkify_Postgres/Sparkify_Postgres


In [6]:
# run the provided function to collect it
song_path = 'data/song_data'
# list of songfiles
song_files = get_files(song_path)
songfile_path = os.path.join(song_path, song_files[0])
# load 1 track into a pandas df
df = pd.read_json(songfile_path, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [8]:
# Select columns
song_features = ['song_id', 'title', 'artist_id', 'year', 'duration']
song_data = df[song_features].values[0].tolist()

In [9]:
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [11]:
cur.execute("rollback")

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [13]:
artist_features = ['artist_id', 'artist_name', 'artist_location',\
                   'artist_latitude', 'artist_longitude']

artist_data = df[artist_features].values[0].tolist()
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
try:
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
!tree

.
├── README.md
├── __init__.py
├── __pycache__
│   ├── sql_queries.cpython-36.pyc
│   └── sql_queries.cpython-39.pyc
├── create_tables.py
├── data
│   ├── log_data
│   │   └── 2018
│   │       └── 11
│   │           ├── 2018-11-01-events.json
│   │           ├── 2018-11-02-events.json
│   │           ├── 2018-11-03-events.json
│   │           ├── 2018-11-04-events.json
│   │           ├── 2018-11-05-events.json
│   │           ├── 2018-11-06-events.json
│   │           ├── 2018-11-07-events.json
│   │           ├── 2018-11-08-events.json
│   │           ├── 2018-11-09-events.json
│   │           ├── 2018-11-10-events.json
│   │           ├── 2018-11-11-events.json
│   │           ├── 2018-11-12-events.json
│   │           ├── 2018-11-13-events.json
│   │           ├── 2018-11-14-events.json
│   │           ├── 2018-11-15-events.json
│   │           ├── 2018-11-16-events.json
│   │           ├── 2018-11-17-events.json
│   │           ├── 2018-11-18-events.json
│   │           ├── 2018-

In [17]:
# run the provided function to collect it
log_path = 'data/log_data/2018/11'
# list of songfiles
log_files = get_files(log_path)
logfile_path = os.path.join(log_path, log_files[0])

# load 1 track into a pandas df
df_log = pd.read_json(logfile_path, lines=True)
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
# Filter NextSong
df_log_filtered = df_log[df_log.page == 'NextSong']
df_log_filtered

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Sidewalk Prophets,Logged In,Molly,F,0,Taylor,260.62322,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,You Love Me Anyway (Album),200,1541970568796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
91,Rise Against,Logged In,Molly,F,1,Taylor,221.17832,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Torches,200,1541970828796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
92,K'Naan,Logged In,Molly,F,2,Taylor,220.49914,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Wavin' Flag,200,1541971049796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
93,Patrick Jumpen,Logged In,Ryan,M,0,Smith,208.87465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,480,Holiday,200,1541979540796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [19]:
df_log_filtered.ts = pd.to_datetime(df_log_filtered.ts,unit='ms')
df_log_filtered.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [20]:
time_data = [df_log_filtered.ts, df_log_filtered.ts.dt.hour, df_log_filtered.ts.dt.day,\
             df_log_filtered.ts.dt.week, df_log_filtered.ts.dt.month, \
            df_log_filtered.ts.dt.year, df_log_filtered.ts.dt.dayofweek]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

/var/folders/6t/tpdh9g_12vl1w4zc123x7_3c0000gn/T/ipykernel_7916/696420383.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_log_filtered.ts.dt.week, df_log_filtered.ts.dt.month, \


In [21]:
len(column_labels)

7

In [22]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-11 02:33:56.796,2,11,45,11,2018,6
1,2018-11-11 02:36:10.796,2,11,45,11,2018,6
2,2018-11-11 02:40:34.796,2,11,45,11,2018,6
4,2018-11-11 04:36:13.796,4,11,45,11,2018,6
5,2018-11-11 04:36:46.796,4,11,45,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df_log_filtered[['userId', 'firstName', 'lastName', 'gender', 'level']]. \
rename(columns={'userId': 'user_id', 'firstName':'first_name', 'lastName': 'last_name'})
 

In [25]:
user_df

,user_id,first_name,last_name,gender,level
0,69,Anabelle,Simpson,F,free
1,69,Anabelle,Simpson,F,free
2,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
...,...,...,...,...,...
90,35,Molly,Taylor,F,free
91,35,Molly,Taylor,F,free
92,35,Molly,Taylor,F,free
93,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [26]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
df_log_filtered.rename(columns={'userId': 'user_id', \
    'firstName':'first_name', 'lastName': 'last_name', \
    'sessionId': 'session_id', 'userAgent' : 'user_agent', 'ts': 'timestamp'}, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [28]:
df_log_filtered

,artist,auth,first_name,gender,itemInSession,last_name,length,level,location,method,page,registration,session_id,song,status,timestamp,user_agent,user_id
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,2018-11-11 02:33:56.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,2018-11-11 02:36:10.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,2018-11-11 02:40:34.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,2018-11-11 04:36:13.796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,2018-11-11 04:36:46.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Sidewalk Prophets,Logged In,Molly,F,0,Taylor,260.62322,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,You Love Me Anyway (Album),200,2018-11-11 21:09:28.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
91,Rise Against,Logged In,Molly,F,1,Taylor,221.17832,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Torches,200,2018-11-11 21:13:48.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
92,K'Naan,Logged In,Molly,F,2,Taylor,220.49914,free,"St. Louis, MO-IL",PUT,NextSong,1.540993e+12,464,Wavin' Flag,200,2018-11-11 21:17:29.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",35
93,Patrick Jumpen,Logged In,Ryan,M,0,Smith,208.87465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,480,Holiday,200,2018-11-11 23:39:00.796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


In [29]:
songplay_features = ['timestamp', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent']

In [30]:
for index, row in df_log_filtered.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        song_id, artist_id = results
    else:
        song_id, artist_id = None, None

    # insert songplay record
    songplay_data = (row.session_id, row.timestamp, row.user_id, row.level, song_id, \
                     artist_id, row.session_id, row.location, row.user_agent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [32]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.